In [16]:
import gzip
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import seaborn as sns
from collections import defaultdict

freebase_path = "/Users/alexdremov/Downloads/freebase-rdf-latest.gz"

In [34]:
char_processed = pd.read_csv(
    "../../data/MovieSummaries/character_processed.csv",
    parse_dates=["actor_date_of_birth", "movie_release_date"],
)

char_processed = char_processed.rename(columns={
    'Wikipedia movie ID': "wikipedia_movie_id",
    'Freebase movie ID': "fb_movie_id",
    'Character name': "character_name",
    'Actor gender': "actor_gender",
    'Actor height (in meters)': "actor_height",
    'Actor ethnicity (Freebase ID)': "fb_actor_eth_id",
    'Actor name': "actor_name",
    'Freebase character/actor map ID': "fb_char_actor_map_id",
    'Freebase character ID': "fb_char_id",
    'Freebase actor ID': "fb_actor_id",
})

freebase_to_name = {
    x['fb_actor_id']: x['actor_name'] for i, x in char_processed.iterrows()
}

char_processed

,wikipedia_movie_id,fb_movie_id,character_name,actor_gender,actor_height,fb_actor_eth_id,actor_name,fb_char_actor_map_id,fb_char_id,fb_actor_id,actor_date_of_birth,movie_release_date,ethn_name,race
0,975900,/m/03vyhn,Akooshay,F,1.620,NaN,Wanda De Jesus,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7,1958-08-26,2001-08-24,NaN,NaN
1,975900,/m/03vyhn,Lieutenant Melanie Ballard,F,1.780,/m/044038p,Natasha Henstridge,/m/0jys3m,/m/0bgchn4,/m/0346l4,1974-08-15,2001-08-24,NaN,NaN
2,975900,/m/03vyhn,Desolation Williams,M,1.727,/m/0x67,Ice Cube,/m/0jys3g,/m/0bgchn_,/m/01vw26l,1969-06-15,2001-08-24,African American,African
3,975900,/m/03vyhn,Sgt Jericho Butler,M,1.750,NaN,Jason Statham,/m/02vchl6,/m/0bgchnq,/m/034hyc,1967-09-12,2001-08-24,NaN,NaN
4,975900,/m/03vyhn,Bashira Kincaid,F,1.650,NaN,Clea DuVall,/m/02vbb3r,/m/0bgchp9,/m/01y9xg,1977-09-25,2001-08-24,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334982,913762,/m/03pcrp,UN Spacy Commander,M,NaN,NaN,Sonny Byrkett,/m/0kr407w,/m/0kr407_,/m/0gn4bz,1954-01-01,1992-05-21,NaN,NaN
334983,913762,/m/03pcrp,Silvie Gena,F,NaN,NaN,Susan Byrkett,/m/0kr40b9,/m/0kr40bf,/m/0gn4nd,1958-01-01,1992-05-21,NaN,NaN
334984,913762,/m/03pcrp,Elensh,F,NaN,NaN,Dorothy Elias-Fahn,/m/0kr406c,/m/0kr406h,/m/0b_vcv,1970-05-01,1992-05-21,NaN,NaN
334985,913762,/m/03pcrp,Hibiki,M,NaN,NaN,Jonathan Fahn,/m/0kr405_,/m/0kr4090,/m/0bx7_j,1965-04-12,1992-05-21,NaN,NaN


In [130]:
len(freebase_to_name)

58176

In [4]:
movies_processed = pd.read_csv(
    "../../data/MovieSummaries/movie_processed.csv",
    parse_dates=["movie_release_date"],
)

movies_processed.rename(columns={
    "Wikipedia movie ID": "wikipedia_movie_id",
    "Freebase movie ID": "fb_movie_id",
    "Movie name": "movie_name",
    "Movie box office revenue": "revenue",
    "Movie runtime": "movie_runtime",
}, inplace=True)
movies_processed

,wikipedia_movie_id,fb_movie_id,movie_name,revenue,movie_runtime,languages,movie_release_date,countries_old,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,['english'],2001-08-24,['United States of America'],['United States of America'],"['Adventure', 'Horror', 'Action', 'Thriller', ..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,NaN,95.0,['english'],2000-02-16,['United States of America'],['United States of America'],"['Crime', 'Drama', 'Mystery', 'Biography']"
2,28463795,/m/0crgdbh,Brun bitter,NaN,83.0,['norwegian'],1988-01-01,['Norway'],['Norway'],"['Crime', 'Drama']"
3,9363483,/m/0285_cd,White Of The Eye,NaN,110.0,['english'],1987-01-01,['United Kingdom'],['United Kingdom'],"['Erotic', 'Thriller']"
4,261236,/m/01mrr1,A Woman in Flames,NaN,106.0,['german'],1983-01-01,['Germany'],['Germany'],['Drama']
...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,NaN,120.0,['english'],2011-03-19,['United States of America'],['United States of America'],['Drama']
81737,34980460,/m/0g4pl34,Knuckle,NaN,96.0,['english'],2011-01-21,"['Ireland', 'United Kingdom']","['Ireland', 'United Kingdom']","['Documentary', 'Drama', 'Biography']"
81738,9971909,/m/02pygw1,Another Nice Mess,NaN,66.0,['english'],1972-09-22,['United States of America'],['United States of America'],"['Comedy', 'Satire']"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,NaN,150.0,['japanese'],1992-05-21,['Japan'],['Japan'],"['Drama', 'Adventure', 'Short Film', 'Cartoon'..."


In [5]:
interesting_ids = (
    set(char_processed.fb_movie_id) |
    set(char_processed.fb_actor_eth_id) |
    set(char_processed.fb_char_actor_map_id) |
    set(char_processed.fb_char_id) |
    set(char_processed.fb_actor_id) |
    set(movies_processed.fb_movie_id)
)
interesting_ids = set([
    '<http://rdf.freebase.com/ns/' + i.lstrip('/').replace('/', '.') + '>'
    for i in interesting_ids
    if isinstance(i, str)
])

len(interesting_ids), next(iter(interesting_ids))

(616271, '<http://rdf.freebase.com/ns/m.02tb3wq>')

In [6]:
import re

def extract_urls(text):
    # Define the regular expression pattern
    pattern = r"<http://rdf\.freebase\.com/ns/[^\s]+>"
    urls = re.findall(pattern, text)
    return urls


extract_urls("sdjkfh <http://rdf.freebase.com/ns/base.type_ontology.non_agent> sdjknfhls")

['<http://rdf.freebase.com/ns/base.type_ontology.non_agent>']

In [7]:
extracted_lines = 0
with gzip.open(freebase_path, 'r') as fin, gzip.open('extracted-all.gz', 'w') as fout:
    progress = tqdm(fin, total=int(3.14 * 1e9))
    for line in progress:
        triplets = line.decode('utf-8').split('\t', 1)
        if triplets[0] in interesting_ids:
            extracted_lines += 1
            progress.set_description(f"{extracted_lines = }", refresh=False)
            fout.write(line)

  0%|          | 0/3140000000 [00:00<?, ?it/s]

In [9]:
single_link_ids = set()

with gzip.open('extracted-all.gz', 'r') as fin:
    progress = tqdm(fin, total=26196949)
    for line in progress:
        triplets = line.decode('utf-8')
        single_link_ids |= set(extract_urls(line.decode('utf-8')))
        progress.set_description(f"{len(single_link_ids) = }", refresh=False)

  0%|          | 0/26196949 [00:00<?, ?it/s]

In [10]:
full_ids = single_link_ids | interesting_ids
len(full_ids)

5328093

In [ ]:
extracted_lines = 0
with gzip.open(freebase_path, 'r') as fin, gzip.open('extracted-all-linked.gz', 'w') as fout:
    progress = tqdm(fin, total=int(3.14 * 1e9), mininterval=2.0)
    for line in progress:
        triplets = line.decode('utf-8').split('\t', 1)
        if triplets[0] in full_ids:
            extracted_lines += 1
            progress.set_description(f"{extracted_lines = }", refresh=False)
            fout.write(line)

  0%|          | 0/3140000000 [00:00<?, ?it/s]

In [32]:
def read_extracted(target, path):
    with gzip.open(path, 'r') as fin:
        for line in tqdm(fin, mininterval=2.0):
            triplets = line.decode('utf-8').split('\t', 1)
            key = triplets[0].lstrip('<http://rdf.freebase.com/ns/').rstrip('>')
            if key not in target:
                target[key] = list()
            target[key].append(triplets[1:])

In [3]:
import gzip, dill

In [24]:
with gzip.open("extracted_freebase.dill.gz", 'r') as file:
    movies_actors = dill.load(file)

In [12]:
person_related_ids = set()

needle = 'people.person.'.encode('utf-8')

with gzip.open('extracted.gz', 'r') as fin:
    progress = tqdm(fin, total=289924448, mininterval=2.0)
    for line in progress:
        if needle in line:
            person_related_ids |= set(extract_urls(line.decode('utf-8')))

  0%|          | 0/289924448 [00:00<?, ?it/s]

In [13]:
len(person_related_ids)

610405

In [14]:
extracted_data = defaultdict(list)

with gzip.open('extracted.gz', 'r') as fin:
    progress = tqdm(fin, total=289924448, mininterval=2.0)
    for line in progress:
        line = line.decode('utf-8')
        triplets = line.split('\t', 2)
        if triplets[0] in person_related_ids:
            progress.set_description(f"{len(extracted_data) = }", refresh=False)
            extracted_data[triplets[0]].append(triplets[1:])

  0%|          | 0/289924448 [00:00<?, ?it/s]

In [8]:
import gzip, dill

with gzip.open("/Users/alexdremov/DocumentsLocal/StudyLocal/epfl/ada/ada-2024-project-cyberbab00shka/src/notebooks/extracted_freebase.dill.gz", 'r') as file:
    movies_actors = dill.load(file)

with gzip.open("/Users/alexdremov/DocumentsLocal/StudyLocal/epfl/ada/ada-2024-project-cyberbab00shka/src/notebooks/extracted_personas.dill.gz", 'r') as file:
    personas = dill.load(file)

In [9]:
def extract_label(entries):
    for predicate, value in entries:
        if predicate == '<http://www.w3.org/2000/01/rdf-schema#label>' and '@en' in value:
            return value.replace('@en', '').strip('\t"\n. ')
    return None

def extract_actor_info(fb_id):
    fb_id = fb_id.strip('/').replace('/', '.')
    fb_id = f"<http://rdf.freebase.com/ns/{fb_id}>"

    if fb_id not in movies_actors:
        return None

    info = movies_actors[fb_id]

    result = dict(
        education=False,
        count_guest_roles=sum(i[0] == '<http://rdf.freebase.com/ns/tv.tv_actor.guest_roles>' for i in info),
        places_lived=sum(i[0] == '<http://rdf.freebase.com/ns/people.person.places_lived>' for i in info),
        spouses=sum(i[0] == '<http://rdf.freebase.com/ns/people.person.spouse_s>' for i in info),
        children=sum(i[0] == '<http://rdf.freebase.com/ns/people.person.children>' for i in info),
        professions_num=sum(i[0] == '<http://rdf.freebase.com/ns/people.person.profession>' for i in info),
        professions=[]
    )

    for predicate, value in info:
        value = value.strip('.\t \n')

        if predicate == '<http://rdf.freebase.com/ns/people.person.date_of_birth>':
            result['date_of_birth'] = value.split("^^")[0].strip('"')

        if predicate == '<http://rdf.freebase.com/ns/people.person.gender>':
            result['gender'] = extract_label(personas[value])

        if predicate == '<http://rdf.freebase.com/ns/people.person.height_meters>':
            result['height'] = value.strip('" .')

        if predicate == '<http://rdf.freebase.com/ns/people.person.nationality>':
            result['nationality'] = extract_label(personas[value])

        if predicate == '<http://rdf.freebase.com/ns/people.person.profession>':
            result['professions'].append(extract_label(personas[value]))

        if predicate == '<http://rdf.freebase.com/ns/people.person.weight_kg>':
            result['weight'] = value.strip('" .')

        if predicate == '<http://rdf.freebase.com/ns/people.person.education>':
            result['education'] = True

        if predicate == '<http://rdf.freebase.com/ns/people.person.place_of_birth>':
            result['place_of_birth'] = extract_label(personas[value])

        if predicate == '<http://rdf.freebase.com/ns/people.person.religion>':
            result['religion'] = extract_label(personas[value])

        if '<http://rdf.freebase.com/ns/common.topic.topic_equivalent_webpage>' in predicate and '<http://en.wikipedia.org/' in value:
            result['wiki_en_page'] = value.strip('<> \n')
        
        if '<http://rdf.freebase.com/key/wikipedia.en_id>' in predicate:
            result['wiki_en_id'] = value.strip('<> \n"')
        
        if '<http://rdf.freebase.com/key/wikipedia.en_title>' in predicate:
            result['wiki_en_title'] = value.strip('<> \n"')
        
        if '<http://rdf.freebase.com/ns/type.object.key>' in predicate and '/wikipedia/en' in value:
            result['wiki_en_path'] = value.strip('<> \n"')
        
        if '<http://rdf.freebase.com/ns/common.topic.description>' in predicate and '@en' in value:
             result['discription'] = value.replace('@en', '').strip('<> \n"')
    return result

In [10]:
all_actors = set(char_processed.fb_actor_id)

In [18]:
extracted_actors_data = dict()

for actor_id in tqdm(all_actors):
    extracted_actors_data[actor_id] = extract_actor_info(actor_id)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58176/58176 [08:16<00:00, 117.15it/s]


In [20]:
# import json
# with open('extracted_actors_data.json', 'w') as file:
#     json.dump(extracted_actors_data, file)

In [21]:
for i, val in extracted_actors_data.items():
    if val is not None:
        val['id'] = i

In [22]:
df = pd.DataFrame(list(filter(lambda x: x is not None, extracted_actors_data.values())))
df

,education,count_guest_roles,places_lived,spouses,children,professions_num,professions,date_of_birth,nationality,gender,place_of_birth,discription,id,wiki_en_path,wiki_en_title,wiki_en_page,wiki_en_id,height,religion,weight
0,False,0,0,0,0,1,[Actor],1970-01-02,Germany,Female,Germany,Chris Hohenester is a German actress.,/m/0j50vrd,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,0,1,0,0,5,"[Screenwriter, Comedian""-GB, Writer, Film Prod...",1960-02-27,United States of America,Female,Circle Pines,"Mary Joseph \""Mary Jo\"" Pehl is an American wr...",/m/03xmr6,/wikipedia/en/Mary_Jo_Pehl,Mary_Jo_Pehl,http://en.wikipedia.org/wiki/index.html?curid=...,995072,NaN,NaN,NaN
2,True,14,1,0,0,5,"[Voice acting""-GB, Actor, Comedian""-GB, Screen...",1978-11-23,United Kingdom,Male,Cricklewood,"Kayvan Novak is a British-Iranian actor, voice...",/m/0gwpbl,/wikipedia/en_id/6912128,Kayvan_Novak,http://en.wikipedia.org/wiki/Kayvan_Novak,6912128,1.85,NaN,NaN
3,False,31,0,0,0,1,[Actor],1912-10-30,England,Male,West Ham,Preston Lockwood was an English actor.\nHe is ...,/m/0ck5l5,/wikipedia/en_id/4724456,Preston_Lockwood,http://en.wikipedia.org/wiki/index.html?curid=...,4724456,NaN,NaN,NaN
4,False,0,0,2,0,1,[Actor],1907-08-09,United States of America,Female,Los Angeles,Marion Burns was an American film actress of t...,/m/02wbw52,/wikipedia/en_id/12532780,Marion_Burns,http://en.wikipedia.org/wiki/Marion_Burns,12532780,1.63,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57571,False,0,1,0,0,1,[Actor],1996-02-19,United States of America,Male,Park Ridge,Allen Alvarado is an American teen actor. He p...,/m/0bfwvk,/wikipedia/en/Allen_Alvarado,Allen_Alvarado,http://en.wikipedia.org/wiki/index.html?curid=...,4056707,NaN,NaN,NaN
57572,False,0,0,2,2,1,[Actor],1919,Greece,Female,Heraklion,Ilya Livykou or Ilia Livykou was a Greek actre...,/m/05f3dq5,/wikipedia/en_id/21389123,Ilya_Livykou,http://en.wikipedia.org/wiki/Ilya_Livykou,21389123,NaN,NaN,NaN
57573,False,0,0,0,3,3,"[Actor, Film director""-GB, Screenwriter]",1962-10-25,France,Female,NaN,"Christine Citti is an actress, film director a...",/m/04p68qc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57574,False,0,0,0,0,1,[Actor],1978-12-19,India,Male,NaN,Hiten Paintal is an actor.,/m/04gl2kv,/wikipedia/en_title/Hiten_Paintal,Hiten_Paintal,http://en.wikipedia.org/wiki/index.html?curid=...,18698581,NaN,NaN,NaN


In [115]:
import requests
from typing import List
from tqdm import trange
import joblib
import wptools

def request_data(title, page=True):
    try:
        so = wptools.page(title, silent=True, ).get_parse()
    except Exception:
        return None
    return so

def get_wikipedia_data(titles: List[int]) -> List[str]:
    result = {}
    batch_size = 50

    titles = [i for i in titles if i is not None and not pd.isna(i)]

    # Process the page IDs in batches
    for data, title in zip(
        joblib.Parallel(return_as='generator', backend='threading', n_jobs=256)(
            (request_data, [title], {}) for title in tqdm(titles)
        ),
        titles
    ):
        result[title] = data
    return result

In [50]:
id_to_wiki_title = get_wikipedia_titles(df.wiki_en_id)



  0%|                                                                                                                                                                                                                        | 0/864 [00:00<?, ?it/s]

  7%|███████████████▎                                                                                                                                                                                              | 64/864 [00:00<00:02, 344.14it/s]

 11%|███████████████████████▌                                                                                                                                                                                      | 99/864 [00:00<00:03, 242.13it/s]

 15%|██████████████████████████████▎                                                                                                                                                                              | 128/864 [00:00<00:03, 206.10it/s]

 19%|█████

In [36]:
df['freebase_to_wiki_title'] = df.apply(
    lambda x: id_to_wiki_title.get(x['wiki_en_id'], freebase_to_name[x['id']]),
    axis=1
)

In [38]:
df

,education,count_guest_roles,places_lived,spouses,children,professions_num,professions,date_of_birth,nationality,gender,...,discription,id,wiki_en_path,wiki_en_title,wiki_en_page,wiki_en_id,height,religion,weight,freebase_to_wiki_title
0,False,0,0,0,0,1,[Actor],1970-01-02,Germany,Female,...,Chris Hohenester is a German actress.,/m/0j50vrd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chris Hohenester
1,False,0,1,0,0,5,"[Screenwriter, Comedian""-GB, Writer, Film Prod...",1960-02-27,United States of America,Female,...,"Mary Joseph \""Mary Jo\"" Pehl is an American wr...",/m/03xmr6,/wikipedia/en/Mary_Jo_Pehl,Mary_Jo_Pehl,http://en.wikipedia.org/wiki/index.html?curid=...,995072,NaN,NaN,NaN,Mary Jo Pehl
2,True,14,1,0,0,5,"[Voice acting""-GB, Actor, Comedian""-GB, Screen...",1978-11-23,United Kingdom,Male,...,"Kayvan Novak is a British-Iranian actor, voice...",/m/0gwpbl,/wikipedia/en_id/6912128,Kayvan_Novak,http://en.wikipedia.org/wiki/Kayvan_Novak,6912128,1.85,NaN,NaN,Kayvan Novak
3,False,31,0,0,0,1,[Actor],1912-10-30,England,Male,...,Preston Lockwood was an English actor.\nHe is ...,/m/0ck5l5,/wikipedia/en_id/4724456,Preston_Lockwood,http://en.wikipedia.org/wiki/index.html?curid=...,4724456,NaN,NaN,NaN,Preston Lockwood
4,False,0,0,2,0,1,[Actor],1907-08-09,United States of America,Female,...,Marion Burns was an American film actress of t...,/m/02wbw52,/wikipedia/en_id/12532780,Marion_Burns,http://en.wikipedia.org/wiki/Marion_Burns,12532780,1.63,NaN,NaN,Marion Burns
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57571,False,0,1,0,0,1,[Actor],1996-02-19,United States of America,Male,...,Allen Alvarado is an American teen actor. He p...,/m/0bfwvk,/wikipedia/en/Allen_Alvarado,Allen_Alvarado,http://en.wikipedia.org/wiki/index.html?curid=...,4056707,NaN,NaN,NaN,None
57572,False,0,0,2,2,1,[Actor],1919,Greece,Female,...,Ilya Livykou or Ilia Livykou was a Greek actre...,/m/05f3dq5,/wikipedia/en_id/21389123,Ilya_Livykou,http://en.wikipedia.org/wiki/Ilya_Livykou,21389123,NaN,NaN,NaN,Ilya Livykou
57573,False,0,0,0,3,3,"[Actor, Film director""-GB, Screenwriter]",1962-10-25,France,Female,...,"Christine Citti is an actress, film director a...",/m/04p68qc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Christine Citti
57574,False,0,0,0,0,1,[Actor],1978-12-19,India,Male,...,Hiten Paintal is an actor.,/m/04gl2kv,/wikipedia/en_title/Hiten_Paintal,Hiten_Paintal,http://en.wikipedia.org/wiki/index.html?curid=...,18698581,NaN,NaN,NaN,Hiten Paintal


In [58]:
merged = pd.merge(char_processed[['fb_actor_id', 'actor_name']].drop_duplicates(), df, left_on='fb_actor_id', right_on='id', how='left')
merged['title_to_search'] = merged.apply(
    lambda x: x['freebase_to_wiki_title'] if isinstance(x['freebase_to_wiki_title'], str) else x['actor_name'],
    axis=1
)
merged

,fb_actor_id,actor_name,education,count_guest_roles,places_lived,spouses,children,professions_num,professions,date_of_birth,...,id,wiki_en_path,wiki_en_title,wiki_en_page,wiki_en_id,height,religion,weight,freebase_to_wiki_title,title_to_search
0,/m/03wcfv7,Wanda De Jesus,False,23.0,1.0,1.0,0.0,2.0,"[Actor, Voice acting""-GB]",1958-08-26,...,/m/03wcfv7,/wikipedia/en_id/16191329,Wanda_De_Jesus,http://en.wikipedia.org/wiki/index.html?curid=...,16191329,1.62,NaN,NaN,Wanda De Jesus,Wanda De Jesus
1,/m/0346l4,Natasha Henstridge,False,28.0,2.0,3.0,2.0,2.0,"[Fashion Model""-GB, Actor]",1974-08-15,...,/m/0346l4,/wikipedia/en_id/704403,Natasha_Henstridge,http://en.wikipedia.org/wiki/index.html?curid=...,704403,1.78,NaN,NaN,Natasha Henstridge,Natasha Henstridge
2,/m/01vw26l,Ice Cube,True,30.0,1.0,1.0,5.0,10.0,"[Screenwriter, Film Score Composer, Voice acti...",1969-06-15,...,/m/01vw26l,/wikipedia/en_title/Ice_Cube,Ice_Cube,http://en.wikipedia.org/wiki/index.html?curid=...,148328,1.727,Islam,NaN,Ice Cube,Ice Cube
3,/m/034hyc,Jason Statham,False,7.0,2.0,2.0,0.0,6.0,"[Actor, Voice acting""-GB, Martial Artist, Dive...",1967-07-26,...,/m/034hyc,/wikipedia/en/Jason_statham,Jason_Statham,http://en.wikipedia.org/wiki/index.html?curid=...,707815,1.78,NaN,77.27,Jason Statham,Jason Statham
4,/m/01y9xg,Clea DuVall,True,21.0,0.0,0.0,0.0,1.0,[Actor],1977-09-25,...,/m/01y9xg,/wikipedia/en_id/342768,Clea_DuVall,http://en.wikipedia.org/wiki/Clea_DuVall,342768,1.65,NaN,NaN,Clea DuVall,Clea DuVall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58171,/m/0hppfjk,Wyn Richmond,False,0.0,0.0,2.0,0.0,1.0,[Actor],1906-05-17,...,/m/0hppfjk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wyn Richmond,Wyn Richmond
58172,/m/0kr4097,Tsutomu Takayama,False,0.0,0.0,0.0,0.0,1.0,"[Voice acting""-GB]",1970-03-01,...,/m/0kr4097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tsutomu Takayama,Tsutomu Takayama
58173,/m/01qwg7h,笠原弘子,False,6.0,0.0,0.0,0.0,3.0,"[Singer""-GB, Actor, Voice acting""-GB]",1970-02-19,...,/m/01qwg7h,/wikipedia/en/Hiroko_Kasahara,Hiroko_Kasahara,http://en.wikipedia.org/wiki/Hiroko_Kasahara,841876,NaN,NaN,NaN,Hiroko Kasahara,Hiroko Kasahara
58174,/m/0gn4bz,Sonny Byrkett,False,0.0,0.0,0.0,0.0,1.0,"[Voice acting""-GB]",1954,...,/m/0gn4bz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sonny Byrkett,Sonny Byrkett


In [116]:
found_info = get_wikipedia_data(merged['title_to_search'])













  0%|                                                                                                                                                                                                                      | 0/58176 [00:00<?, ?it/s]API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}












  1%|█▊                                                                                                                                                                                                        | 512/58176 [00:00<00:28, 2038.27it/s]API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/ap

In [121]:
import dill

with open('found_info.dill', 'wb') as file:
    dill.dump(found_info, file, )

In [125]:
infoboxes = {k: i.data['infobox'] for k, i in found_info.items() if i is not None}

In [128]:
merged_dicts = merged.to_dict('records')

for i in merged_dicts:
    infobox = infoboxes.get(i['title_to_search'])
    if infobox is None:
        continue
    i |= {
        f"infobox_{k}": v for k, v in infobox.items()
    }

In [129]:
merged_infobox = pd.DataFrame(merged_dicts)
merged_infobox

,fb_actor_id,actor_name,education,count_guest_roles,places_lived,spouses,children,professions_num,professions,date_of_birth,...,infobox_armaments,infobox_placeofburial,infobox_placeofburial_label,infobox_artist,infobox_editor,infobox_sooyears1,infobox_sooteam1,infobox_soogames_goals1,infobox_servicenumber,infobox_nationalteam-update
0,/m/03wcfv7,Wanda De Jesus,False,23.0,1.0,1.0,0.0,2.0,"[Actor, Voice acting""-GB]",1958-08-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/m/0346l4,Natasha Henstridge,False,28.0,2.0,3.0,2.0,2.0,"[Fashion Model""-GB, Actor]",1974-08-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/m/01vw26l,Ice Cube,True,30.0,1.0,1.0,5.0,10.0,"[Screenwriter, Film Score Composer, Voice acti...",1969-06-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/m/034hyc,Jason Statham,False,7.0,2.0,2.0,0.0,6.0,"[Actor, Voice acting""-GB, Martial Artist, Dive...",1967-07-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/m/01y9xg,Clea DuVall,True,21.0,0.0,0.0,0.0,1.0,[Actor],1977-09-25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58171,/m/0hppfjk,Wyn Richmond,False,0.0,0.0,2.0,0.0,1.0,[Actor],1906-05-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58172,/m/0kr4097,Tsutomu Takayama,False,0.0,0.0,0.0,0.0,1.0,"[Voice acting""-GB]",1970-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58173,/m/01qwg7h,笠原弘子,False,6.0,0.0,0.0,0.0,3.0,"[Singer""-GB, Actor, Voice acting""-GB]",1970-02-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58174,/m/0gn4bz,Sonny Byrkett,False,0.0,0.0,0.0,0.0,1.0,"[Voice acting""-GB]",1954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df.to_csv('actors_freebase.csv')

In [131]:
merged_infobox.to_csv('actors_freebase_wiki.csv')

In [50]:
all_chars = set(char_processed.fb_char_id) | set(char_processed.fb_char_actor_map_id)

interesting_ids = set([
    '<http://rdf.freebase.com/ns/' + i.lstrip('/').replace('/', '.') + '>'
    for i in all_chars
    if isinstance(i, str)
])

In [55]:
extracted_data = defaultdict(list)

with gzip.open('extracted.gz', 'r') as fin:
    progress = tqdm(fin, total=289924448, mininterval=2.0)
    for line in progress:
        line = line.decode('utf-8')
        triplets = line.split('\t', 2)
        if triplets[0] in interesting_ids:
            progress.set_description(f"{len(extracted_data) = }", refresh=False)
            extracted_data[triplets[0]].append(triplets[1:])

  0%|          | 0/289924448 [00:00<?, ?it/s]

In [56]:
# with gzip.open("extracted_chars.dill.gz", 'w') as file:
#     dill.dump(extracted_data, file)

In [57]:
with gzip.open("extracted_chars.dill.gz", 'r') as file:
    chars_data = dill.load(file)

In [73]:
def extract_character_info(fb_id):
    fb_id = fb_id.strip('/').replace('/', '.')
    fb_id = f"<http://rdf.freebase.com/ns/{fb_id}>"

    if fb_id not in movies_actors:
        return None

    info = chars_data[fb_id]

    result = dict()

    for predicate, value in info:
        value: str = value.strip('\t.\n ')
        print(predicate, value)
        if predicate == '<http://rdf.freebase.com/ns/common.topic.description>' and value.endswith('@en'):
            result['description'] = value.replace('@en', '').strip('".')
    return result